# Import Libraries

In [1]:
import rioxarray as rio
import os
from dask.distributed import Client, LocalCluster, Lock
import xarray as xr
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sensus import *
from xrspatial import zonal_stats
import pandas as pd
from shapely.geometry import shape
import geopandas as gpd
import json


## Set Data Path and Set Band Mapper

In [4]:
output_dir="output"



multib_dir="../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC"
lst_dir="../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_TIR25cm"

ortho_files=[f"{multib_dir}/{x}" for x in os.listdir(multib_dir) if x.split(".")[-1] == "tif"]
ndvi_files=[f"{output_dir}/{x}" for x in os.listdir("output") if "ndvi" in x]


dates=[file.split("/")[-1].split("_")[0] for file in ortho_files]

lst_files=[f"{lst_dir}/{x}" for x in os.listdir(lst_dir) if x.split(".")[-1] == "tif" and "CropRGB" in x]



# # Create output
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)



bands=["R","G","B","RC","NIR"]

dataset=dict()

for file in ortho_files:
    date=file.split("/")[-1].split("_")[0]
    dataset[date] ={}
    dataset[date]["ortho"] = file


for file in lst_files:
    date=file.split("/")[-1].split("_")[0]
    dataset[date].update({"lst": file})

for file in ndvi_files:
    date=file.split("/")[-1].split("_")[0]
    dataset[date].update({"ndvi": file})

    

## Prepare Dask Client

In [2]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='10GB')


# Calculate NDVI and Downsample to 25cm


In [5]:


TARGET_RESOLUTION=0.25 # Set target resolution to 25 cm


dates=list(dataset.keys())
data_vals=list()


for date in dates:
    paths=dataset[date]

    ndvi_out=f"{output_dir}/{date}_ndvi_25cm.tif"
    print(date, paths["ortho"])
    # Read Ortho as DaskArray
    ortho=rio.open_rasterio(paths["ortho"], chunks=True,  lock=Lock("rio-read")) # Read ortho as DaskArray
    fill_value=ortho.attrs["_FillValue"]
    ortho=ortho.assign_coords(band=bands) # Assign band names to orthomosaic - 1 = R, 2 = G, 3 = B, 4 = RC, 5 = NIR

    if "long_name" in ortho.attrs:
        del ortho.attrs["long_name"]
        del ortho.attrs["units"]

    # Retrieve NIR Band (5)
    nir=ortho.sel(band="NIR")
    nir=nir.where(nir != fill_value)

    # Retrieve Calibrated Red Band (4)
    red=ortho.sel(band="RC")
    red=red.where(red != fill_value)
    ndvi_5cm = ndvi(nir,red)
    
    ndvi_5cm.attrs=ortho.attrs # Pass original Orthomosaic Attributes to derived NDVI DataArray
    print(f"{date} NDVI Calculation Complete")
    ortho.close()

    ndvi_5cm=ndvi_5cm.expand_dims(dim="band",axis=0) # Reshape array to (band, height, width)
    ndvi_5cm=ndvi_5cm.assign_coords(band=["NDVI"]) # Rename band to NDVI

    
    # Write NDVI, Snow and Vegetation Rasters (25 cm)
    if os.path.exists(ndvi_out) == False:
        
        ndvi_25cm=resample_da(ndvi_5cm, target_res=TARGET_RESOLUTION) # Resamle NDVI Layer from 5cm to 25cm
        ndvi_5cm.close() 

        ndvi_25cm.rio.to_raster(ndvi_out) # Write resampled NDVI Raster to GeoTIFF



20170621 ../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC/20170621_MultiB_RGBNIR.tif
20170621 NDVI Calculation Complete
Resampling input raster to 25.0 cm resolution
20170627 ../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC/20170627_MultiB_RGBNIR.tif
20170627 NDVI Calculation Complete
Resampling input raster to 25.0 cm resolution
20170705 ../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC/20170705_MultiB_RGBNIR.tif
20170705 NDVI Calculation Complete
Resampling input raster to 25.0 cm resolution
20170711 ../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC/20170711_MultiB_RGBNIR.tif
20170711 NDVI Calculation Complete
Resampling input raster to 25.0 cm resolution
20170718 ../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC/20170718_MultiB_RGBNIR.tif
20170718 NDVI Calculation Complete
Resampling input raster to 25.0 cm resolution
20170725 ../../data/UAV_Mapping_Niwot_Ridge_Colorado_2021/UAV_MULTISPEC/20170725_MultiB_RGBNIR.tif
2

## Classify Snow, Dense Vegetation and Sparse Vegetation Based on NDVI



The Normalized Difference Vegetation Index (NDVI) is a widely used indicator for assessing vegetation health and density. NDVI values range from -1.0 to 1.0, with different ranges corresponding to various surface features and vegetation types:

- Negative values: Indicate non-vegetated surfaces such as water bodies, clouds, or snow.
- Values close to 0: Represent barren areas like rocks, sand, or snow.
- Low positive values (approximately 0.2 to 0.3): Correspond to shrublands and grasslands.
- Moderate values (around 0.3 to 0.5): Indicate areas with sparse vegetation or transitional zones.
- High values (approximately 0.6 to 0.8): Signify dense vegetation, such as temperate and tropical forests.

[Source: NASA Earth Observatory](https://earthobservatory.nasa.gov/features/MeasuringVegetationh?)


In [ ]:
CLASSES=["Dense Vegetation","Moderate Vegetation","Sparse Vegetation","Snow"]
VALUES=range(1, len(CLASSES) + 1)
CLASSES=dict(zip(CLASSES,VALUES))
DENSE_VEGETATION_NDVI=[0.6,1]
MODERATE_VEGETATION_NDVI=[0.3,0.6]
SPARSE_VEGETATION_NDVI= [0.2,0.3]
SNOW_NDVI=[-1,0.2]


ndvi_files=[f"{output_dir}/{x}" for x in os.listdir("output") if "ndvi" in x]
# array=np.full((orig_shape), 0)


for ndvi_file in ndvi_files:   

    
    date=ndvi_file.split("/")[-1].split("_")[0]
    lc_out=f"{output_dir}/{date}_lc_25cm.tif"
    print(date)
    ndvi_25cm=rio.open_rasterio(ndvi_file)

    dense=ndvi_25cm.where((ndvi_25cm >= DENSE_VEGETATION_NDVI[0]))
    moderate=ndvi_25cm.where((ndvi_25cm >= SPARSE_VEGETATION_NDVI[0])  & (ndvi_25cm < SPARSE_VEGETATION_NDVI[1])) 
    sparse=ndvi_25cm.where((ndvi_25cm >= MODERATE_VEGETATION_NDVI[0])  & (ndvi_25cm < MODERATE_VEGETATION_NDVI[1])) 
    snow=ndvi_25cm.where((ndvi_25cm < SNOW_NDVI[1])) 

    # Fill Masked layers with their corresponding class values as defined in the CLASSES list

    ## DENSE VEGETATION LAYER
    dense=dense.where(dense.isnull(),CLASSES.get("Dense Vegetation")).where(~dense.isnull(),0)
    
    ## MODERATE VEGETATION CLASS LAYER
    moderate=moderate.where(moderate.isnull(),CLASSES.get("Moderate Vegetation")).where(~moderate.isnull(),0)

    ## SPARSE VEGETATION LAYER
    sparse=sparse.where(sparse.isnull(),CLASSES.get("Sparse Vegetation")).where(~sparse.isnull(),0)

    ## SNOW LAYER
    snow=snow.where(snow.isnull(), CLASSES.get("Snow")).where(~snow.isnull(),0)

    # LAND COVER LAYER - Aggregate all four land classification layers to create a single land cover raster dataset
    lc = sum([snow,dense, moderate, sparse])
    lc=lc.where(lc > 0) # Mask out 0 (FillValue)

      # Write NDVI, Snow and Vegetation Rasters (25 cm)
    # if os.path.exists(lc_out) == False:    
    lc.rio.to_raster(lc_out) # Export as GeoTIFF

# Export class mapping as JSON
with open("classes.json","w") as json_out:

    json_out.write(json.dumps(CLASSES))



20170621
20170627
20170705
20170711
